In [1]:
import pickle
import pandas as pd

In [2]:
with open("df_all_cleaned.pickle", "rb") as read_file:
    df_all = pickle.load(read_file)

In [3]:
from p05_nlp import vectorize_tfidf
from p05_nlp import reduce_dim_lsa
from p05_nlp import reduce_dim_lda
from p05_nlp import reduce_dim_nmf
from p05_nlp import display_topics

In [4]:
import re

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

In [188]:
import seaborn as sns

# Indicate pd series to model

In [6]:
df_all.head()

,title,year,director,reviews_list,imdb_url,wiki_page,url,poster_url,reviews_clean
0,200 Motels,1971,Tony Palmer,"There is no film quite like 200 Motels, but a ...",https://www.imdb.com/title/tt0066732/,https://en.wikipedia.org/wiki/200_Motels,/wiki/200_Motels,upload.wikimedia.org/wikipedia/en/thumb/8/85/2...,film like motel lot strange appearance e...
1,2001: A Space Odyssey,1968,Stanley Kubrick,"A stand-alone monument in cinema history, Stan...",https://www.imdb.com/title/tt0062622/,https://en.wikipedia.org/wiki/2001:_A_Space_Od...,/wiki/2001:_A_Space_Odyssey_(film),upload.wikimedia.org/wikipedia/en/thumb/9/94/S...,stand monument cinema history stanley kubric...
2,37°2 le Matin (Betty Blue),1986,Jean-Jacques Beineix,The Betty of the title is like a shooting star...,https://www.imdb.com/title/tt0090563/,https://en.wikipedia.org/wiki/Betty_Blue,/wiki/Betty_Blue,upload.wikimedia.org/wikipedia/en/thumb/9/9d/B...,betty title like shoot star run hot bright ...
3,42nd Street,1933,Lloyd Bacon,Most of the negative comments posted below see...,https://www.imdb.com/title/tt0024034/,https://en.wikipedia.org/wiki/42nd_Street_(film),/wiki/42nd_Street_(film),upload.wikimedia.org/wikipedia/en/thumb/8/8b/F...,negative comment post people don t like musica...
4,The 7th Voyage of Sinbad,1957,Nathan H. Juran,"Arguably, Harryhausen's finest moment. I can't...",https://www.imdb.com/title/tt0051337/,https://en.wikipedia.org/wiki/The_7th_Voyage_o...,/wiki/The_7th_Voyage_of_Sinbad,upload.wikimedia.org/wikipedia/en/thumb/a/a6/S...,arguably harryhausen fine moment t head no...


In [7]:
reviews = df_all['reviews_clean'].copy()
type(reviews)

pandas.core.series.Series

In [8]:
df_all.iloc[1159]

title                      Poultrygeist: Night of the Chicken Dead
year                                                          2006
director                                             Lloyd Kaufman
reviews_list     It was disgusting, perverse, and childish, and...
imdb_url                     https://www.imdb.com/title/tt0462485/
wiki_page        https://en.wikipedia.org/wiki/Poultrygeist:_Ni...
url                  /wiki/Poultrygeist:_Night_of_the_Chicken_Dead
poster_url       upload.wikimedia.org/wikipedia/en/thumb/6/6c/W...
reviews_clean    disgusting   perverse   childish   love minute...
Name: 1159, dtype: object

In [83]:
df_all[df_all['reviews_list'].isna()]

,title,year,director,reviews_list,imdb_url,wiki_page,url,poster_url,reviews_clean


# Additional text cleaning steps

Remove names

In [10]:
names = ['allen','woody','bakshi','thomerson',
         'jack','corman','roger','karloff',
         'stanley','kubrick','harryhausen',
         'tim','helen','megan','lena','mel',
         'brooks','werner','herzog', 'william',
         'wyler','matthew','hopkins','harry',
         'bromley','davenport','dorothy','judy',
         'garland','warhol','leone','sandler',
         'adam','carpenter','warwick','davis',
         'de','john','davis','woo','kevin',
         'smith','fonda','jay','russell',
         'cronenberg','bergman','reeve','lynch',
         'palma','max','diane','cusack',
         'michael','myers','ad','th','kieslowski',
         'aniston','meyer','billy','jennifer',
         'troma','van','damme', 'jean','bruce',
         'lee','charles','george','romero',
         'tarantino','trancer','deth','verhoeven',
         'polanski','les','michel','moer','slater']
rem_names =  lambda x: re.sub("|".join(names), "", x)
reviews = reviews.map(rem_names)
            

Correct typos

In [11]:
reviews = reviews.map(lambda x: re.sub('pyon', 'python', x))


Remove duplicate whitespaces

In [12]:
# result = re.sub(r'\s+', ' ',   input)           # Eliminate duplicate whitespaces
rem_empty_space =  lambda x: re.sub(r'\s+', ' ', x)
reviews = reviews.map(rem_empty_space)

Create custom bi-grams

In [13]:
reviews = reviews.map(lambda x: re.sub('monty python', 'monty_python', x))
reviews = reviews.map(lambda x: re.sub('bruce lee', 'bruce_lee', x))
reviews = reviews.map(lambda x: re.sub('sci fi', 'sci_fi', x))
reviews = reviews.map(lambda x: re.sub('kung fu', 'kung_fu', x))
reviews = reviews.map(lambda x: re.sub('spaghetti western', 'spaghetti_western', x))


In [14]:
reviews

0       film like motel lot strange appearance especia...
1       stand monument cinema history magnum opus spac...
2       betty title like shoot star run hot bright bur...
3       negative comment post people don t like musica...
4       arguably fine moment t he nominate well ventur...
5       morn man recall love relationship case song sh...
6       film isn t people lot movie general course par...
7       film sensational infamous clockwork orange imp...
8       don t blame state unrstand superlative surroun...
9       love ia coffin joe joseph grave love output lo...
10      probably good horror comedy doesn t fun monste...
11      dortmund mining company go shut young people u...
12      film probably horrible take seriously fortunat...
13      m guess everybody know story movie opportunity...
14       diction original vampire movie fact t ink lik...
15      people plot sense prevertesque way mainly link...
16      wasn t feeling good today know finally get cha...
17      ve re 

In [15]:
reviews[1159]

'disgusting perverse childish love minute inclu e want row longe fan like ia musical zombie chicken nudity gore satire movie minute liver movie hour good movie ve see acting actually good don t wait come vio find eater near show drive hour regret ank lloyd kaufman ank ank ank poultrygeist night chicken late offering easily good movie lo make awesome copious gore breast sex bian political incorrectness scatological humour fat people slime car crash mention gore sex poultrygeist tell wonrful story american chicken bunker fast food store build ancient indian burial ground hawk tribe precise young arbie ci work fy ex girlfriend bian follow typical style employee fend hor zombie chicken want human fh lloyd kaufman return toxic avenger root go gross comedy musical horror fun hard love consistently funny way excellent musical number truly disgusting special effect poultrygeist prove morally incorrect satire alive kick fact movie like meet spartan eatrical release doesn t prove need far superi

# Vectorize with Tf-Idf

In [16]:
reviews_sparse_matrix_lsa, reviews_feature_names_lsa = vectorize_tfidf(reviews, min_df=7, max_df=0.8)

       aa       aan  aaron        ab     aback   abandon  abandonment  \
0     0.0  0.000000    0.0  0.000000  0.000000  0.000000          0.0   
1     0.0  0.000000    0.0  0.000000  0.000000  0.000000          0.0   
2     0.0  0.000000    0.0  0.000000  0.000000  0.000000          0.0   
3     0.0  0.000000    0.0  0.000000  0.000000  0.004240          0.0   
4     0.0  0.000000    0.0  0.000000  0.000000  0.000000          0.0   
5     0.0  0.000000    0.0  0.000000  0.000000  0.000000          0.0   
6     0.0  0.000000    0.0  0.000000  0.000000  0.000000          0.0   
7     0.0  0.000000    0.0  0.000000  0.000000  0.000000          0.0   
8     0.0  0.000000    0.0  0.016822  0.000000  0.007254          0.0   
9     0.0  0.000000    0.0  0.000000  0.000000  0.000000          0.0   
10    0.0  0.000000    0.0  0.000000  0.000000  0.000000          0.0   
11    0.0  0.000000    0.0  0.000000  0.000000  0.000000          0.0   
12    0.0  0.000000    0.0  0.000000  0.000000  0.0

# Reduce dimensions with LSA

In [17]:
reviews_lsa, lsa_components = reduce_dim_lsa(reviews_sparse_matrix_lsa, 14)

In [18]:
topic_names = ['Horror Comedy','Gory','Sci Fi / Space Monster','Anime',
               'Silly','Animated Monster Concert','Martial Arts',
               'Music Documentary',"Creature Feature",'Foreign Monsters',
               'Sex Monsters','Noir / Classic','Violence',
               'Family']
#                 ,'Monsters On Drugs',
#                'Ridiculous', 'Holiday Monsters','Bollywood',
#                'Holiday Space Race','Holiday Monsters','Spaghetti Western']#, 21, 22,'Foreign Silliness', 'British']
display_topics(lsa_components, reviews_feature_names_lsa, 20, topic_names=topic_names)


Topic: ' Horror Comedy '
horror, comedy, funny, woman, action, girl, effect, fun, laugh, sex, monster, kill, kid, alien, animation, amp, black, song, vampire, art

Topic: ' Gory '
horror, vampire, zombie, monster, gore, alien, dracula, effect, scary, blood, creature, halloween, slasher, creepy, scare, sci_fi, frankenstein, budget, dr, hammer

Topic: ' Sci Fi / Space Monster '
animation, alien, monster, sci_fi, planet, animate, space, disney, science, anime, creature, giant, robot, effect, cartoon, ear, godzilla, japanese, scientist, ship

Topic: ' Anime '
vampire, animation, anime, dracula, animate, hammer, disney, cartoon, art, martial, japanese, count, kung_fu, carmilla, beautiful, action, nosferatu, version, fantasy, hunter

Topic: ' Silly '
comedy, vampire, funny, laugh, joke, hilarious, humor, fun, monster, gag, dracula, alien, funniest, song, kid, sketch, spoof, python, monty_python, russian

Topic: ' Animated Monster Concert '
animation, zombie, animate, horror, disney, anime, 

In [19]:
# reviews_sparse_matrix_lsa, reviews_feature_names_lsa = vectorize_tfidf(reviews, min_df=7, max_df=0.8)
# reviews_lsa, lsa_components = reduce_dim_lsa(reviews_sparse_matrix_lsa, 14)
# topic_names = ['Horror Comedy','Gory','Space Monster','Anime',
#                'Silly','Animated Monster Concert','Kung Fu',
#                'Music Documentary',"Creature Feature",'Foreign Monsters',
#                'Sex Monsters','Noir','Soviet',
#                'British']
# #                 ,'Monsters On Drugs',
# #                'Ridiculous', 'Holiday Monsters','Bollywood',
# #                'Holiday Space Race','Holiday Monsters','Spaghetti Western']#, 21, 22,'Foreign Silliness', 'British']
# display_topics(lsa_components, reviews_feature_names_lsa, 15, topic_names=topic_names)

# Topic: ' Horror Comedy '
# horror, comedy, funny, woman, music, action, girl, effect, kill, fun, laugh, sex, monster, kid, friend

# Topic: ' Gory '
# horror, vampire, zombie, monster, gore, alien, dracula, creature, effect, blood, scary, sci_fi, halloween, creepy, slasher

# Topic: ' Space Monster '
# alien, monster, animation, sci_fi, planet, space, science, creature, effect, ear, robot, scientist, giant, godzilla, ship

# Topic: ' Anime '
# vampire, animation, anime, dracula, disney, planet, cartoon, japanese, art, sci_fi, action, martial, hammer, version, fantasy

# Topic: ' Silly '
# comedy, vampire, funny, laugh, joke, hilarious, humor, fun, monster, gag, dracula, song, alien, funniest, kid

# Topic: ' Animated Monster Concert '
# animation, zombie, horror, band, anime, song, music, disney, cartoon, rock, concert, punk, musical, beatle, gore

# Topic: ' Kung Fu '
# martial, action, kung_fu, fight, japanese, kong, hong, dragon, zombie, anime, kill, animation, cop, violence, chinese

# Topic: ' Music Documentary '
# band, music, rock, concert, punk, song, monster, martial, drug, beatle, documentary, musical, album, footage, gang

# Topic: ' Creature Feature '
# zombie, alien, russian, sci_fi, planet, soviet, band, vampire, polish, space, science, ear, russia, fiction, war

# Topic: ' Foreign Monsters '
# monster, russian, soviet, zombie, godzilla, polish, martial, russia, frankenstein, japanese, dracula, poland, kung_fu, art, french

# Topic: ' Sex Monsters '
# sex, zombie, gay, porn, sexual, japanese, monster, woman, rape, anime, godzilla, erotic, martial, explicit, art

# Topic: ' Noir '
# zombie, monster, animation, noir, western, frankenstein, dracula, cop, gang, gangster, crime, lugosi, police, godzilla, italian

# Topic: ' Soviet '
# russian, soviet, polish, poland, russia, gang, sex, anime, monster, animation, rape, violence, punk, cop, communist

# Topic: ' British '
# polish, poland, band, killer, documentary, horror, python, concert, cop, funny, leprechaun, beatle, sketch, monty_python, joke

In [20]:
# Topic: ' Horror Comedy '
# horror, comedy, funny, world, woman, music, performance, action, girl, effect, kill, fun, guy, young, laugh

# Topic: ' Gory '
# horror, vampire, monster, zombie, alien, gore, dracula, creature, effect, blood, scary, sci_fi, frankenstein, halloween, scare

# Topic: ' Space Monster '
# alien, monster, animation, sci_fi, planet, space, science, creature, godzilla, effect, giant, ear, robot, scientist, ship

# Topic: ' Anime '
# vampire, animation, anime, dracula, japanese, art, disney, planet, cartoon, world, beautiful, miyazaki, sci_fi, visual, action

# Topic: ' Silly '
# vampire, comedy, funny, laugh, dracula, joke, hilarious, humor, fun, gag, song, monster, funniest, kid, sketch

# Topic: ' Monster Concert '
# animation, zombie, horror, band, disney, anime, song, music, cartoon, rock, concert, punk, gore, musical, cat

# Topic: ' Kung Fu '
# martial, action, kung_fu, fight, japanese, kong, hong, dragon, kill, anime, violence, cop, gang, killer, chinese

# Topic: ' Music Documentary '
# band, music, rock, concert, monster, song, punk, beatle, drug, documentary, musical, album, footage, martial, musician

# Topic: ' Creature Feature '
# monster, godzilla, frankenstein, creature, giant, animation, kid, dracula, beast, japanese, child, girl, school, baron, toho

# Topic: ' Soviet Monsters '
# zombie, monster, russian, soviet, polish, martial, godzilla, french, japanese, poland, kung_fu, russia, band, art, comedy

# Topic: ' Sex Monsters '
# sex, gay, porn, zombie, sexual, japanese, woman, rape, anime, monster, alien, erotic, drug, martial, godzilla

# Topic: ' Camp '
# zombie, animation, monster, noir, cop, western, frankenstein, gangster, dracula, gang, crime, police, godzilla, drug, lugosi

# Topic: ' Schoolkid Martial Artist '
# zombie, school, girl, kid, teen, child, martial, disney, book, family, boy, young, moer, christmas, kung_fu

# Topic: ' Schoolkid Soviet Gangster '
# russian, soviet, russia, gang, girl, school, sex, rape, teen, moscow, western, polish, woman, kid, violence

# Reduce Dimensions with LDA

In [21]:
# reviews_sparse_matrix_lda, reviews_feature_names_lda = vectorize_tfidf(reviews, min_df=2, max_df=0.9)

In [22]:
# reviews_lda, lda_components = reduce_dim_lda(reviews_sparse_matrix_lda, 10)

In [23]:
# display_topics(lda_components, reviews_feature_names_lda, 15)

# Reduce Dimensions with NMF

Use NMF so that topic values are interpretable.

In [24]:
reviews_sparse_matrix_nmf, reviews_feature_names_nmf = vectorize_tfidf(reviews, min_df=2, max_df=0.9, print_matrix=False)

In [48]:
reviews_nmf, nmf_components = reduce_dim_nmf(reviews_sparse_matrix_nmf, num_topics=16, random_state=42)

In [162]:
nmf_topics = ["Art/Classic","Gory/Scary", "Sci Fi", "Vampires", "Silly", "Animated", "Martial Arts", "Music Documentary", 
              "Living Dead", "Giant Monster", "Russia", "Sex", "Gold", "Action", "Kids", "Holiday",]
display_topics(nmf_components, reviews_feature_names_nmf, num_top_words=15, topic_names=nmf_topics)


Topic: ' Art/Classic '
french, art, cinema, war, image, camera, leave, experience, book, beautiful, dream, woman, mind, documentary, reality

Topic: ' Gory/Scary '
horror, gore, slasher, halloween, house, scary, killer, scare, craven, creepy, murr, blood, kill, effect, ghost

Topic: ' Sci Fi '
alien, sci_fi, planet, space, science, ear, ship, effect, mars, fiction, robot, spaceship, special, scientist, budget

Topic: ' Vampires '
vampire, dracula, horror, hammer, blood, count, carmilla, cushing, castle, martin, nosferatu, helse, kronos, sarandon, susan

Topic: ' Silly '
comedy, funny, laugh, joke, humor, hilarious, gag, fun, sketch, python, funniest, guy, stupid, monty_python, humour

Topic: ' Animated '
animation, animate, anime, disney, cartoon, cat, fritz, voice, japanese, ralph, ult, animated, wizard, fantasy, rabbit

Topic: ' Martial Arts '
martial, kung_fu, fight, action, kong, art, hong, dragon, chinese, japanese, master, chow, fighting, tournament, fighter

Topic: ' Music Docu

In [104]:
# nmf_topics = ["Art","Gory", "Sci Fi", "Vampires", "Silly", "Animated", "Martial Arts", "Music Documentary", 
#               "Living Dead", "Giant Monster", "Soviet", "Sex", "Gold", "Action", "Kids"]

# Topic  Art
# world, art, french, cinema, war, image, camera, leave, book, experience, long, dream, beautiful, reality, mind

# Topic  1
# horror, gore, slasher, halloween, scary, house, killer, scare, craven, kill, creepy, murr, blood, effect, ghost

# Topic  2
# alien, sci_fi, planet, space, science, ear, ship, effect, mars, fiction, robot, spaceship, scientist, special, budget

# Topic  3
# vampire, dracula, horror, hammer, blood, count, helse, carmilla, castle, martin, nosferatu, cushing, kronos, sarandon, susan

# Topic  4
# comedy, funny, laugh, joke, humor, hilarious, gag, fun, sketch, python, funniest, guy, monty_python, stupid, comic

# Topic  5
# animation, anime, disney, cartoon, japanese, fritz, cat, voice, ralph, wizard, ult, miyazaki, fantasy, version, akira

# Topic  6
# martial, kung_fu, fight, action, art, kong, dragon, hong, chinese, japanese, master, chow, tournament, fighter, fist

# Topic  7
# band, music, rock, concert, song, beatle, punk, musical, documentary, album, stone, fan, musician, zappa, metal

# Topic  8
# zombie, horror, gore, shaun, corpse, dawn, live, lugosi, night, budget, cemetery, rob, lionel, fulci, moss

# Topic  9
# monster, godzilla, creature, frankenstein, giant, dracula, beast, scientist, japanese, island, effect, horror, dr, gamera, attack

# Topic  10
# russian, soviet, polish, russia, poland, communist, moscow, katya, comedy, communism, union, regime, leningr, dza, ussr

# Topic  11
# sex, woman, porn, gay, sexual, rape, erotic, girl, bian, male, female, explicit, relationship, nudity, violence

# Topic  12
# leprechaun, lep, gold, cody, vegas, bridget, hood, horror, rap, space, coin, clover, series, mack, las

# Topic  13
# action, cop, gang, western, kill, violence, noir, police, killer, crime, car, gangster, performance, gun, guy

# Topic  14
# school, kid, teen, girl, high, christmas, santa, teacher, child, teenager, boy, young, parent, stunt, friend

# Find Cosine Similarity

## NMF

In [105]:
reviews_nmf_df = pd.DataFrame(reviews_nmf)
reviews_nmf_df.shape

(1674, 16)

In [106]:
reviews_nmf_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.061364,0.041946,0.048288,0.000000,0.039165,0.081423,0.007300,0.991167,0.015822,0.000000,0.000000,0.025206,0.000000,0.025092,0.000000,0.000000
1,0.697145,0.000000,0.713602,0.000000,0.000000,0.033641,0.056692,0.007784,0.000000,0.000000,0.000000,0.000000,0.018837,0.000000,0.000000,0.000000
2,0.845127,0.000000,0.000000,0.000000,0.142109,0.071227,0.000000,0.008262,0.000000,0.000000,0.000000,0.510286,0.000000,0.000000,0.005641,0.000000
3,0.144939,0.012531,0.000000,0.000000,0.251689,0.000000,0.034609,0.761048,0.000000,0.038882,0.000000,0.081232,0.000000,0.279925,0.497673,0.030806
4,0.130296,0.000000,0.241058,0.000000,0.000000,0.488226,0.435024,0.000000,0.002372,0.674507,0.000000,0.000000,0.081455,0.069978,0.124394,0.123823
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.341137,0.000000,0.000000,0.000000,0.940013,0.000000,0.000000,0.000000,0.000000
6,0.685460,0.000000,0.059520,0.000000,0.082518,0.231110,0.456444,0.000000,0.028778,0.171933,0.022268,0.036714,0.014719,0.474964,0.000000,0.000000
7,0.779012,0.044571,0.054158,0.000000,0.000000,0.024746,0.004789,0.038046,0.000000,0.000000,0.187640,0.356379,0.009444,0.459823,0.111343,0.000000
8,0.986443,0.000000,0.000000,0.000000,0.000000,0.000000,0.019211,0.075534,0.000000,0.000000,0.000000,0.021803,0.001784,0.142748,0.000000,0.000000
9,0.094890,0.895227,0.121031,0.106919,0.108791,0.000000,0.000000,0.010625,0.136864,0.013297,0.000000,0.281220,0.018158,0.230680,0.000000,0.000000


In [107]:
df_columns

['title',
 'Art',
 'Gory',
 'Sci Fi',
 'Vampires',
 'Silly',
 'Animated',
 'Martial Arts',
 'Music Documentary',
 'Living Dead',
 'Giant Monster',
 'Russia',
 'Sex',
 'Gold',
 'Action',
 'Kids',
 'Holiday']

In [108]:
df_columns = ['title'] + nmf_topics
df_topics = pd.concat([df_all['title'],reviews_nmf_df], axis=1)
df_topics.columns=df_columns

In [109]:
df_topics.shape

(1674, 17)

In [110]:
plan_9_mask = df_topics['title']=='Plan 9 from Outer Space'
df_topics[plan_9_mask]

,title,Art/Classic,Gory/Scary,Sci Fi,Vampires,Silly,Animated,Martial Arts,Music Documentary,Living Dead,Giant Monster,Russia,Sex,Gold,Action,Kids,Holiday
1665,Plan 9 from Outer Space,0.187048,0.021237,0.873272,0.072457,0.349102,0.0,0.0,0.054511,0.246873,0.069341,0.0,0.0,0.003336,0.078094,0.0,0.0


In [111]:
df_topics[plan_9_mask].index

Int64Index([1665], dtype='int64')

In [112]:
plan_9_sim = cosine_similarity(reviews_nmf_df[~plan_9_mask],reviews_nmf_df[plan_9_mask])

In [113]:
# df_topics.iloc[1159]
# df_topics[df_topics['title']=='Manos: The Hands of Fate']
df_topics[df_topics['title']=='Manos: The Hands of Fate'].index

Int64Index([966], dtype='int64')

In [114]:
plan_9_sim_df = pd.concat([df_all['title'],pd.DataFrame(plan_9_sim)], axis=1)
plan_9_sim_df.rename(columns={'title':'title', 0:'similarity'}, inplace=True)
plan_9_sim_df

,title,similarity
0,200 Motels,0.128104
1,2001: A Space Odyssey,0.754056
2,37°2 le Matin (Betty Blue),0.208140
3,42nd Street,0.181284
4,The 7th Voyage of Sinbad,0.287974
5,9 Songs,0.018596
6,300,0.265166
7,A Clockwork Orange,0.231969
8,À Bout de Souffle (Breathless),0.199784
9,À Meia-Noite Levarei Sua Alma (At Midnight I'l...,0.241544


In [115]:
plan_9_sim_df.sort_values(by='similarity',ascending=False).head(5)

,title,similarity
923,The Lost Skeleton of Cadavra,0.947318
792,Jigureul Jikyeora! (Save the Green Planet!),0.947314
357,Dark Star,0.930946
110,Battlefield Earth,0.929512
214,The Brother from Another Planet,0.924923


In [116]:
manos_mask = df_topics['title']=='Manos: The Hands of Fate'
manos_sim = cosine_similarity(reviews_nmf_df[~manos_mask],reviews_nmf_df[manos_mask])
df_topics[manos_mask]

,title,Art/Classic,Gory/Scary,Sci Fi,Vampires,Silly,Animated,Martial Arts,Music Documentary,Living Dead,Giant Monster,Russia,Sex,Gold,Action,Kids,Holiday
966,Manos: The Hands of Fate,0.70655,0.486625,0.166725,0.053876,0.378573,0.0,0.127978,0.129582,0.067848,0.22491,0.0,0.032752,0.006397,0.02191,0.003832,0.0


In [117]:
manos_sim_df = pd.concat([df_all['title'],pd.DataFrame(manos_sim)], axis=1)
manos_sim_df.rename(columns={'title':'title', 0:'similarity'}, inplace=True)
manos_sim_df

,title,similarity
0,200 Motels,0.218466
1,2001: A Space Odyssey,0.619927
2,37°2 le Matin (Betty Blue),0.668728
3,42nd Street,0.326280
4,The 7th Voyage of Sinbad,0.342320
5,9 Songs,0.074992
6,300,0.636214
7,A Clockwork Orange,0.608906
8,À Bout de Souffle (Breathless),0.713071
9,À Meia-Noite Levarei Sua Alma (At Midnight I'l...,0.597843


In [118]:
manos_sim_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1674 entries, 0 to 1673
Data columns (total 2 columns):
title         1674 non-null object
similarity    1673 non-null float64
dtypes: float64(1), object(1)
memory usage: 26.2+ KB


In [119]:
manos_sim_df['similarity'].max()

0.9224597104186463

In [120]:
manos_sim_df.sort_values(by='similarity',ascending=False).head(5)

,title,similarity
344,A Cure for Wellness,0.922460
1516,Troll 2,0.919267
367,The Dead Father,0.911508
748,Incubus,0.895574
796,John Dies at the End,0.895337


In [121]:
manos_sim_df[manos_sim_df['similarity']>0.9]

,title,similarity
344,A Cure for Wellness,0.922460
367,The Dead Father,0.911508
1516,Troll 2,0.919267


# Write a generic recommendation function (no topic selection)

In [189]:
def recommender(movie):
    print("Movies most similar to '{}':".format(movie))
    movie_mask = df_topics['title']== movie
    movie_sim = cosine_similarity(reviews_nmf_df[~movie_mask],reviews_nmf_df[movie_mask])
    movie_sim_df = pd.concat([df_all['title'],pd.DataFrame(movie_sim)], axis=1)
    movie_sim_df.rename(columns={'title':'title', 0:'similarity'}, inplace=True)
    return movie_sim_df.sort_values(by='similarity',ascending=False).head(5)

In [190]:
sample_movie = "Ziggy Stardust and the Spiders from Mars"
sample_movie_2 = "The Princess Bride"
sample_movie_3 = "Shaun of the Dead"

In [195]:
movie_sim_df = recommender(sample_movie)
movie_sim_df

Movies most similar to 'Ziggy Stardust and the Spiders from Mars':


,title,similarity
1264,Rude Boy,0.988174
1537,Urgh! A Music War,0.987541
1370,The Song Remains the Same,0.987449
879,The Last Waltz,0.987449
591,Gimme Shelter,0.987052


In [206]:
df_topics.iloc[movie_sim_df.index]

,title,Art/Classic,Gory/Scary,Sci Fi,Vampires,Silly,Animated,Martial Arts,Music Documentary,Living Dead,Giant Monster,Russia,Sex,Gold,Action,Kids,Holiday
1264,Rude Boy,0.000000,0.000000,0.008175,0.000000,0.0,0.000000,0.00000,0.999475,0.001179,0.007929,0.007624,0.019315,0.0,0.022092,0.000000,0.000000
1537,Urgh! A Music War,0.000000,0.000000,0.011136,0.000303,0.0,0.024986,0.00095,0.998365,0.016199,0.000000,0.000000,0.007316,0.0,0.034477,0.031285,0.005929
1370,The Song Remains the Same,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
879,The Last Waltz,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
591,Gimme Shelter,0.027818,0.005529,0.000000,0.000000,0.0,0.000000,0.00000,0.999597,0.000000,0.000000,0.000000,0.000000,0.0,0.001037,0.000000,0.000000


In [198]:
df_topics[df_topics['title']==sample_movie]

,title,Art/Classic,Gory/Scary,Sci Fi,Vampires,Silly,Animated,Martial Arts,Music Documentary,Living Dead,Giant Monster,Russia,Sex,Gold,Action,Kids,Holiday
1668,Ziggy Stardust and the Spiders from Mars,0.0,0.0,0.152114,0.042163,0.0,0.0,0.0,0.987449,0.0,0.0,0.0,0.0,0.005213,0.0,0.0,0.0


In [202]:
pd.concat(df_topics[df_topics['title']==sample_movie], movie_sim_df)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [192]:
recommender(sample_movie_2)

Movies most similar to 'The Princess Bride':


,title,similarity
1618,Wild Style,0.903304
79,Babe: Pig in the City,0.878522
121,Bedtime for Bonzo,0.866280
1475,Tiān Xià Dì Yī Quán (also known as Five Finger...,0.859318
1621,Willard,0.852856


In [193]:
recommender(sample_movie_3)

Movies most similar to 'Shaun of the Dead':


,title,similarity
199,Braindead,0.947799
1229,The Return of the Living Dead,0.947115
515,Fido,0.946370
1669,Zodiac,0.909248
362,Day of the Dead,0.906806


In [187]:
df_topics[df_topics['title']==sample_movie_3]

,title,Art/Classic,Gory/Scary,Sci Fi,Vampires,Silly,Animated,Martial Arts,Music Documentary,Living Dead,Giant Monster,Russia,Sex,Gold,Action,Kids,Holiday
1319,Shaun of the Dead,0.0,0.012183,0.012514,0.00467,0.41586,0.0,0.0,0.0,0.909248,0.0,0.0,0.0,0.000938,0.0,0.0,0.0


# Topic Slicing

## Figure out the process

In [122]:
df_topics[manos_mask]

,title,Art/Classic,Gory/Scary,Sci Fi,Vampires,Silly,Animated,Martial Arts,Music Documentary,Living Dead,Giant Monster,Russia,Sex,Gold,Action,Kids,Holiday
966,Manos: The Hands of Fate,0.70655,0.486625,0.166725,0.053876,0.378573,0.0,0.127978,0.129582,0.067848,0.22491,0.0,0.032752,0.006397,0.02191,0.003832,0.0


In [133]:
topic_slice_manos

,title,Silly,Sci Fi,Giant Monster
966,Manos: The Hands of Fate,0.378573,0.166725,0.22491


In [141]:
column_slice = ['title','Silly','Sci Fi','Giant Monster']
topic_slice_manos = df_topics.loc[manos_mask,column_slice]
topic_slice_not_manos = df_topics.loc[~manos_mask,column_slice]
topic_slice_not_manos

,title,Silly,Sci Fi,Giant Monster
0,200 Motels,0.039165,0.048288,0.000000
1,2001: A Space Odyssey,0.000000,0.713602,0.000000
2,37°2 le Matin (Betty Blue),0.142109,0.000000,0.000000
3,42nd Street,0.251689,0.000000,0.038882
4,The 7th Voyage of Sinbad,0.000000,0.241058,0.674507
5,9 Songs,0.000000,0.000000,0.000000
6,300,0.082518,0.059520,0.171933
7,A Clockwork Orange,0.000000,0.054158,0.000000
8,À Bout de Souffle (Breathless),0.000000,0.000000,0.000000
9,À Meia-Noite Levarei Sua Alma (At Midnight I'l...,0.108791,0.121031,0.013297


In [145]:
reviews_nmf_df[manos_mask]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
966,0.70655,0.486625,0.166725,0.053876,0.378573,0.0,0.127978,0.129582,0.067848,0.22491,0.0,0.032752,0.006397,0.02191,0.003832,0.0


In [155]:
manos_sim_topic_slice = cosine_similarity(topic_slice_not_manos.iloc[:,1:],topic_slice_manos.iloc[:,1:])
topic_slice_df = pd.concat([df_all['title'],pd.DataFrame(manos_sim_topic_slice)], axis=1)
topic_slice_df.rename(columns={'title':'title', 0:'similarity'}, inplace=True)
topic_slice_df

,title,similarity
0,200 Motels,0.781481
1,2001: A Space Odyssey,0.354094
2,37°2 le Matin (Betty Blue),0.804021
3,42nd Street,0.867523
4,The 7th Voyage of Sinbad,0.568973
5,9 Songs,0.000000
6,300,0.848670
7,A Clockwork Orange,0.354094
8,À Bout de Souffle (Breathless),0.000000
9,À Meia-Noite Levarei Sua Alma (At Midnight I'l...,0.837070


In [156]:
topic_slice_df.sort_values(by='similarity',ascending=False).head(5)

,title,similarity
566,Game of Death,0.999736
999,Miracle Mile,0.998712
972,Mars Attacks!,0.998464
390,Demonlover,0.998290
168,Blonde Venus,0.997841


In [157]:
column_slice += ['Martial Arts']
topic_slice_manos = df_topics.loc[manos_mask,column_slice]
topic_slice_not_manos = df_topics.loc[~manos_mask,column_slice]
topic_slice_not_manos

,title,Silly,Sci Fi,Giant Monster,Martial Arts
0,200 Motels,0.039165,0.048288,0.000000,0.007300
1,2001: A Space Odyssey,0.000000,0.713602,0.000000,0.056692
2,37°2 le Matin (Betty Blue),0.142109,0.000000,0.000000,0.000000
3,42nd Street,0.251689,0.000000,0.038882,0.034609
4,The 7th Voyage of Sinbad,0.000000,0.241058,0.674507,0.435024
5,9 Songs,0.000000,0.000000,0.000000,0.000000
6,300,0.082518,0.059520,0.171933,0.456444
7,A Clockwork Orange,0.000000,0.054158,0.000000,0.004789
8,À Bout de Souffle (Breathless),0.000000,0.000000,0.000000,0.019211
9,À Meia-Noite Levarei Sua Alma (At Midnight I'l...,0.108791,0.121031,0.013297,0.000000


In [158]:
manos_sim_topic_slice = cosine_similarity(topic_slice_not_manos.iloc[:,1:],topic_slice_manos.iloc[:,1:])
topic_slice_df = pd.concat([df_all['title'],pd.DataFrame(manos_sim_topic_slice)], axis=1)
topic_slice_df.rename(columns={'title':'title', 0:'similarity'}, inplace=True)
topic_slice_df.sort_values(by='similarity',ascending=False).head(5)

,title,similarity
999,Miracle Mile,0.998774
1317,Shao Lin san Shi liu Fang (The 36th Chamber of...,0.991190
152,Birdemic: Shock and Terror,0.986273
378,Death Race 2000,0.982263
1654,Alvin Purple,0.982071


## Write the Function

In [163]:
df_topics.columns

Index(['title', 'Art/Classic', 'Gory/Scary', 'Sci Fi', 'Vampires', 'Silly',
       'Animated', 'Martial Arts', 'Music Documentary', 'Living Dead',
       'Giant Monster', 'Russia', 'Sex', 'Gold', 'Action', 'Kids', 'Holiday'],
      dtype='object')

In [174]:
sample_movie = "Ziggy Stardust and the Spiders from Mars"
sample_movie_2 = "The Princess Bride"
sample_movie_3 = "Shaun of the Dead"
sample_topics = ['Silly','Gory/Scary','Living Dead','Sci Fi','Music Documentary','Art/Classic']
def similarity_by_slices(movie, topics_list):
    # create movie mask on df_topics
    movie_mask = df_topics['title']== movie
    # generate movie topic sliders
    topic_dist = df_topics[movie_mask]
    # add topics_list to columns_slice
    columns_slice = ['title'] + topics_list
    # create other masks
    topic_slice_movie = df_topics.loc[movie_mask,columns_slice]
    topic_slice_not_movie = df_topics.loc[~movie_mask,columns_slice]
    # generate cosine similarity and create dataframe
    movie_sim_topic_slice = cosine_similarity(topic_slice_not_movie.iloc[:,1:],topic_slice_movie.iloc[:,1:])
    topic_slice_df = pd.concat([df_all['title'],pd.DataFrame(movie_sim_topic_slice)], axis=1)
    topic_slice_df.rename(columns={'title':'title', 0:'similarity'}, inplace=True)
    
    
    print(topic_slice_df.sort_values(by='similarity',ascending=False).head(5))


In [175]:
similarity_by_slices(sample_movie, sample_topics)

                                                  title  similarity
1585                                       The Warriors    0.999322
978                     Master of the Flying Guillotine    0.999146
751   Intāsutera Fō Faibu (Interstella 5555: The 5to...    0.998759
0                                            200 Motels    0.990909
1537                                  Urgh! A Music War    0.989848


# Different type of function

In [207]:
df_topics.columns

Index(['title', 'Art/Classic', 'Gory/Scary', 'Sci Fi', 'Vampires', 'Silly',
       'Animated', 'Martial Arts', 'Music Documentary', 'Living Dead',
       'Giant Monster', 'Russia', 'Sex', 'Gold', 'Action', 'Kids', 'Holiday'],
      dtype='object')

In [219]:
dict_in = {'Gory/Scary': [0.012183], "Silly": [0.41586], "Living Dead": [0.909248]}

In [ ]:
# column_slice = ['title','Silly','Sci Fi','Giant Monster']
# topic_slice_manos = df_topics.loc[manos_mask,column_slice]
# topic_slice_not_manos = df_topics.loc[~manos_mask,column_slice]

In [224]:
def topic_levels(dict_in):
    # create df_in from dict_in
    df_in = pd.DataFrame.from_dict(dict_in)
    topics = df_in.columns
    # get slice of df_topics
    df_slice = df_topics.loc[:,topics]
    # do cosine similarity
    movie_sim = cosine_similarity(df_slice,df_in)
    # get recommendations
    movie_rec = pd.concat([df_all['title'],pd.DataFrame(movie_sim)], axis=1)
    movie_rec.rename(columns={'title':'title', 0:'similarity'}, inplace=True)
    #return list of movies
    return movie_rec.sort_values(by='similarity',ascending=False).head(5)
    

In [225]:
topic_levels(dict_in)

,title,similarity
1319,Shaun of the Dead,1.000000
579,Gekitotsu! Satsujin Ken (The Street Fighter),0.999693
730,I'm Not There,0.999323
309,Coffy,0.998167
747,The Incredibly Strange Creatures Who Stopped L...,0.990777


In [227]:
df_topics.iloc[747][0]

'The Incredibly Strange Creatures Who Stopped Living And Became Mixed-Up Zombies'

# Sliders widget

In [248]:
import ipywidgets as widgets
from IPython.display import display
 
topics = df_topics.columns #['horror', 'comedy', 'romance']
sliders={}
style = {'description_width': 'initial', 'widget_width': 'initial'}
# IntSlider(description='A too long description', style=style)
for top in topics[1:]:
    sliders[top] = widgets.IntSlider(value=10, description=top, style=style)
    display(sliders[top])


IntSlider(value=10, description='Art/Classic', style=SliderStyle(description_width='initial'))

IntSlider(value=10, description='Gory/Scary', style=SliderStyle(description_width='initial'))

IntSlider(value=10, description='Sci Fi', style=SliderStyle(description_width='initial'))

IntSlider(value=10, description='Vampires', style=SliderStyle(description_width='initial'))

IntSlider(value=10, description='Silly', style=SliderStyle(description_width='initial'))

...

IntSlider(value=10, description='Sex', style=SliderStyle(description_width='initial'))

IntSlider(value=10, description='Gold', style=SliderStyle(description_width='initial'))

IntSlider(value=10, description='Action', style=SliderStyle(description_width='initial'))

IntSlider(value=10, description='Kids', style=SliderStyle(description_width='initial'))

IntSlider(value=10, description='Holiday', style=SliderStyle(description_width='initial'))

In [246]:
for top in topics[1:]:
    print(sliders[top].value/100)

0.1
0.33
0.34
0.32
0.39
0.1
0.62
0.39
0.1
0.1
0.91
0.1
0.1
0.1
0.1
0.1


In [231]:
btn.keys

['_dom_classes',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_view_count',
 '_view_module',
 '_view_module_version',
 '_view_name',
 'continuous_update',
 'description',
 'description_tooltip',
 'disabled',
 'layout',
 'max',
 'min',
 'orientation',
 'readout',
 'readout_format',
 'step',
 'style',
 'value']

# Pickle stuff for next notebook

In [232]:
with open("df_topics.pickle", "wb") as to_write:
    pickle.dump(df_topics, to_write)

In [233]:
with open("reviews_nmf_df.pickle", "wb") as to_write:
    pickle.dump(reviews_nmf_df, to_write)

In [234]:
with open("nmf_topics.pickle", "wb") as to_write:
    pickle.dump(nmf_topics, to_write)

In [235]:
with open("nmf_components.pickle", "wb") as to_write:
    pickle.dump(nmf_components, to_write)

In [236]:
with open("reviews_feature_names_nmf.pickle", "wb") as to_write:
    pickle.dump(reviews_feature_names_nmf, to_write)